
###Во оваа лабораторика вежба фокусот е на time series предвидувања, односно предвидување на Close (BTC) вредноста во даденото податочно множество за Bitcoin. Ваша задача е со помош на decision tree и boosting методи да ја предвидите бараната вредност земајќи ги во предвид релевантните колони од податочното множество, но дополнително и вредноста на Close (BTC) во изминатите четири дена, бидејќи вредноста на Bitcoin-от во даден момент зависи од нејзината вредност во изминатите денови.

Hint : внимавајте како ќе го поделите множеството на train и test бидејќи се работи податоци во временска серија

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Симнете го податочното множество

In [2]:
!gdown 14YlgjT_M8kDrvDKEv864fs7ZyI1lMErO

Downloading...
From: https://drive.google.com/uc?id=14YlgjT_M8kDrvDKEv864fs7ZyI1lMErO
To: /content/crypto_data.csv
100% 90.0k/90.0k [00:00<00:00, 97.1MB/s]


In [3]:
import pandas as pd

In [4]:
df=pd.read_csv('/content/crypto_data.csv')

In [5]:
df

,Date,Close (BNB),Close (BTC),Close (USDT),Close (ETH)
0,09-11-17,1.99077,7143.580078,1.00818,320.884003
1,10-11-17,1.79684,6618.140137,1.00601,299.252991
2,11-11-17,1.67047,6357.600098,1.00899,314.681000
3,12-11-17,1.51969,5950.069824,1.01247,307.907990
4,13-11-17,1.68662,6559.490234,1.00935,316.716003
...,...,...,...,...,...
1748,23-08-22,299.03000,21528.090000,1.00000,1662.770000
1749,24-08-22,296.45000,21395.020000,1.00000,1657.060000
1750,25-08-22,301.58000,21600.900000,1.00010,1696.460000
1751,26-08-22,279.60000,20260.020000,1.00000,1507.780000


In [6]:
df.isnull().sum()

Date            0
Close (BNB)     0
Close (BTC)     0
Close (USDT)    0
Close (ETH)     0
dtype: int64

In [7]:
df.columns

Index(['Date', 'Close (BNB)', 'Close (BTC)', 'Close (USDT)', 'Close (ETH)'], dtype='object')

In [8]:
features =['Close (BNB)', 'Close (BTC)', 'Close (USDT)', 'Close (ETH)']

##Направете претпроцесирање на податоците

In [9]:
df_copy=df.copy()

In [10]:
for column in features:
    df[column+'_1'] = df[column].shift(periods=1)
    df[column+'_2'] = df[column].shift(periods=2)
    df[column+'_3'] = df[column].shift(periods=3)
df

,Date,Close (BNB),Close (BTC),Close (USDT),Close (ETH),Close (BNB)_1,Close (BNB)_2,Close (BNB)_3,Close (BTC)_1,Close (BTC)_2,Close (BTC)_3,Close (USDT)_1,Close (USDT)_2,Close (USDT)_3,Close (ETH)_1,Close (ETH)_2,Close (ETH)_3
0,09-11-17,1.99077,7143.580078,1.00818,320.884003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10-11-17,1.79684,6618.140137,1.00601,299.252991,1.99077,NaN,NaN,7143.580078,NaN,NaN,1.00818,NaN,NaN,320.884003,NaN,NaN
2,11-11-17,1.67047,6357.600098,1.00899,314.681000,1.79684,1.99077,NaN,6618.140137,7143.580078,NaN,1.00601,1.00818,NaN,299.252991,320.884003,NaN
3,12-11-17,1.51969,5950.069824,1.01247,307.907990,1.67047,1.79684,1.99077,6357.600098,6618.140137,7143.580078,1.00899,1.00601,1.00818,314.681000,299.252991,320.884003
4,13-11-17,1.68662,6559.490234,1.00935,316.716003,1.51969,1.67047,1.79684,5950.069824,6357.600098,6618.140137,1.01247,1.00899,1.00601,307.907990,314.681000,299.252991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1748,23-08-22,299.03000,21528.090000,1.00000,1662.770000,300.56000,302.06000,283.69000,21398.910000,21534.120000,21166.060000,1.00000,1.00000,1.00000,1622.510000,1619.320000,1577.000000
1749,24-08-22,296.45000,21395.020000,1.00000,1657.060000,299.03000,300.56000,302.06000,21528.090000,21398.910000,21534.120000,1.00000,1.00000,1.00000,1662.770000,1622.510000,1619.320000
1750,25-08-22,301.58000,21600.900000,1.00010,1696.460000,296.45000,299.03000,300.56000,21395.020000,21528.090000,21398.910000,1.00000,1.00000,1.00000,1657.060000,1662.770000,1622.510000
1751,26-08-22,279.60000,20260.020000,1.00000,1507.780000,301.58000,296.45000,299.03000,21600.900000,21395.020000,21528.090000,1.00010,1.00000,1.00000,1696.460000,1657.060000,1662.770000


In [11]:
df = df.dropna(axis=0).reset_index()


In [12]:
df

,index,Date,Close (BNB),Close (BTC),Close (USDT),Close (ETH),Close (BNB)_1,Close (BNB)_2,Close (BNB)_3,Close (BTC)_1,Close (BTC)_2,Close (BTC)_3,Close (USDT)_1,Close (USDT)_2,Close (USDT)_3,Close (ETH)_1,Close (ETH)_2,Close (ETH)_3
0,3,12-11-17,1.51969,5950.069824,1.01247,307.907990,1.67047,1.79684,1.99077,6357.600098,6618.140137,7143.580078,1.00899,1.00601,1.00818,314.681000,299.252991,320.884003
1,4,13-11-17,1.68662,6559.490234,1.00935,316.716003,1.51969,1.67047,1.79684,5950.069824,6357.600098,6618.140137,1.01247,1.00899,1.00601,307.907990,314.681000,299.252991
2,5,14-11-17,1.59258,6635.750000,1.00683,337.631012,1.68662,1.51969,1.67047,6559.490234,5950.069824,6357.600098,1.00935,1.01247,1.00899,316.716003,307.907990,314.681000
3,6,15-11-17,1.53045,7315.540039,1.00318,333.356995,1.59258,1.68662,1.51969,6635.750000,6559.490234,5950.069824,1.00683,1.00935,1.01247,337.631012,316.716003,307.907990
4,7,16-11-17,1.57792,7871.689941,1.00212,330.924011,1.53045,1.59258,1.68662,7315.540039,6635.750000,6559.490234,1.00318,1.00683,1.00935,333.356995,337.631012,316.716003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1745,1748,23-08-22,299.03000,21528.090000,1.00000,1662.770000,300.56000,302.06000,283.69000,21398.910000,21534.120000,21166.060000,1.00000,1.00000,1.00000,1622.510000,1619.320000,1577.000000
1746,1749,24-08-22,296.45000,21395.020000,1.00000,1657.060000,299.03000,300.56000,302.06000,21528.090000,21398.910000,21534.120000,1.00000,1.00000,1.00000,1662.770000,1622.510000,1619.320000
1747,1750,25-08-22,301.58000,21600.900000,1.00010,1696.460000,296.45000,299.03000,300.56000,21395.020000,21528.090000,21398.910000,1.00000,1.00000,1.00000,1657.060000,1662.770000,1622.510000
1748,1751,26-08-22,279.60000,20260.020000,1.00000,1507.780000,301.58000,296.45000,299.03000,21600.900000,21395.020000,21528.090000,1.00010,1.00000,1.00000,1696.460000,1657.060000,1662.770000


##Поделето го на тренирачко и тестирачко множество во сооднос 80:20

In [13]:
df.columns

Index(['index', 'Date', 'Close (BNB)', 'Close (BTC)', 'Close (USDT)',
       'Close (ETH)', 'Close (BNB)_1', 'Close (BNB)_2', 'Close (BNB)_3',
       'Close (BTC)_1', 'Close (BTC)_2', 'Close (BTC)_3', 'Close (USDT)_1',
       'Close (USDT)_2', 'Close (USDT)_3', 'Close (ETH)_1', 'Close (ETH)_2',
       'Close (ETH)_3'],
      dtype='object')

In [14]:
features=['Close (BNB)_1', 'Close (BNB)_2', 'Close (BNB)_3',
       'Close (BTC)_1', 'Close (BTC)_2', 'Close (BTC)_3', 'Close (USDT)_1',
       'Close (USDT)_2', 'Close (USDT)_3', 'Close (ETH)_1', 'Close (ETH)_2',
       'Close (ETH)_3']

In [15]:
X,Y=df[features],df['Close (BTC)']

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,shuffle=False)

##Искористете соодветен Boosting модел за проблемот и тестирајте го моделот на тестирачкото множество

In [34]:
from xgboost import XGBRegressor

model = XGBRegressor(objective ='reg:linear',colsample_bytree=0.3,learning_rate=0.1,max_depth=15,alpha=0.01,n_estimators=10)
model.fit(X_train, Y_train)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [17:49:56] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


XGBRegressor(alpha=0.01, base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.3, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=15, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=10, n_jobs=None,
             num_parallel_tree=None, ...)

In [35]:
y_pred2 = model.predict(X_test)

In [36]:
from sklearn.metrics import r2_score

In [37]:
print(r2_score(Y_test,y_pred2))

0.32249995126559494


##Искористете соодветен Decision Tree модел за проблемот и тестирајте го моделот на тестирачкото множество

In [38]:
from sklearn.tree import DecisionTreeRegressor
model=DecisionTreeRegressor(max_depth=15)

In [39]:
model.fit(X_train,Y_train)

DecisionTreeRegressor(max_depth=15)

In [41]:
y_pred3=model.predict(X_test)

In [42]:
print(r2_score(Y_test,y_pred3))

0.9293395733607139


##Споредете ги резултатите добиени од двата модели со соодветни метрики

##Bonus Task: Направете визуелизации кои го илустрираат процесот на предвидување на двата горенавередени модели